# Used to help intrepet the output from the cpac outline

In [1]:
import os
import pickle
import numpy as np
from subprocess import call

import sys
sys.path.append('../src/features')

from subject import Subject

cur_dir = os.getcwd()
asd_diag_dir = os.path.dirname(cur_dir)
data_dir = asd_diag_dir + '/data/'
# cpac_output_dir = data_dir + 'cpac_output_t/'
cpac_output_dir = data_dir + 'cpac_output/output/'
subjects_dir = os.path.dirname(cur_dir) + '/data/ABIDEI_subjects/'
trs_save_file = save_dir = os.path.dirname(cur_dir) + '/data/dicts/ABIDEI_site_trs.json'

In [3]:
sub_number = 51471
subject_folder = cpac_output_dir + f'/pipeline_abide_cpac_run_freq-filter_nuisance/sub-00{sub_number}_ses-1/'

In [4]:
# Find all cc200 rois within output
# folder_queue = os.listdir(cpac_output_dir)
# folder_queue = [os.path.join(cpac_output_dir, f) for f in folder_queue]
folder_queue = os.listdir(subject_folder)
folder_queue = [os.path.join(subject_folder, f) for f in folder_queue]
cc_200_l = list()

while len(folder_queue) > 0:
    f = folder_queue.pop(0)
    if(os.path.exists(os.path.join(f, 'roi_stats.npz'))):
        cc_200_l.append(f)
    for d in os.listdir(f):
        if os.path.isdir(os.path.join(f, d)):
            folder_queue.append(os.path.join(f, d))

In [5]:
len(cc_200_l)

54

In [6]:
for d in cc_200_l:
    print(d[200:])
    print()

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_DKT_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..DKT_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Juelich_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Juelich_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC400_mask_file_..cpac_templates..CC400.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Brodmann_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Brodmann_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_rois_3mm_mask_file_..cpac_templates..rois_3mm.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Slab907_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Slab907_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Yeo-7-liberal_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Yeo-7-liberal_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Scha

In [7]:
cc_200_l[1]

'/home/cparish/Capstone/asd_diagnosis_fmri/data/cpac_output/output//pipeline_abide_cpac_run_freq-filter_nuisance/sub-0051471_ses-1/roi_timeseries/_scan_rest_run-1/_selector_CSF-2mmE-M_aC-CSF+WM-2mm-DPC5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Juelich_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Juelich_space-MNI152NLin6_res-1x1x1.nii.gz'

# Analyze ROI output from cpac 

In [8]:
# Load cpac rois
cpac_rois = {}
for roi_folder in cc_200_l:
    call(['unzip', os.path.join(roi_folder, 'roi_stats.npz'), '-d', roi_folder])
    # first row is nans and skip first 5
    roi = np.load(os.path.join(roi_folder, 'arr_0.npy'), allow_pickle=True)[5:] 
    cpac_rois[roi_folder] = roi
    

In [9]:
len(cpac_rois)

54

In [2]:
# Load ABIDEI preprocessed rois by loading subjects
def open_pickle(f):
    file = open(f,'rb')
    o = pickle.load(file)
    file.close()
    return o

def load_subjects_d(subject_folder):
    subjects_d = {}
    for f in os.listdir(subject_folder):
        s = open_pickle(os.path.join(subject_folder, f))
        subjects_d[s._sub_id] = s
    return subjects_d

In [8]:
subjects_d = load_subjects_d(subjects_dir)

In [12]:
s = subjects_d[51471]

In [13]:
cc_200_rois = {}
for folder, roi in cpac_rois.items():
    if roi.shape == (146, 200) and folder.find('CC200') != -1:
        cc_200_rois[folder] = roi

In [14]:
len(cc_200_rois)

2

In [15]:
s._data_dict.keys()

dict_keys(['filt_noglobal_roi_200_Cradd', 'cpac_filt_noglobal'])

In [16]:
goal_roi = np.array(s._data_dict['filt_noglobal_roi_200_Cradd'])
goal_ts = goal_roi[3]
print(goal_ts.shape)

(200,)


In [17]:
min_indexes = list()
for folder, roi in cc_200_rois.items():
    print(f'{folder}:{np.linalg.norm(roi-goal_roi)}')
    min_index = 0
    min_dist = 200
    for i, ts in enumerate(roi):
        dist = np.linalg.norm(ts-goal_ts)
        if dist < min_dist:
            min_index = i
            min_dist = dist
        if(ts == goal_ts).all():
            print('d')
    min_indexes.append(min_index)
#     if(roi[5:] == goal_roi).all():
#         print(roi.shape)

/home/cparish/Capstone/asd_diagnosis_fmri/data/cpac_output/output//pipeline_abide_cpac_run_freq-filter_nuisance/sub-0051471_ses-1/roi_timeseries/_scan_rest_run-1/_selector_CSF-2mmE-M_aC-CSF+WM-2mm-DPC5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz:1367.8197454035635
/home/cparish/Capstone/asd_diagnosis_fmri/data/cpac_output/output//pipeline_abide_cpac_run_freq-filter_nuisance/sub-0051471_ses-1/roi_timeseries/_scan_rest_run-1/_selector_CSF-2mmE-M_aC-CSF+WM-2mm-DPC5_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz:1208.4897610950802


In [ ]:
np.array(s._data_dict['filt_noglobal_roi_200_Cradd']).shape

In [ ]:
goal_ts

In [ ]:
for f in cc_200_rois.keys():
    print(f.split('/')[-1])

# Analyzing with Subjects

In [213]:
# for filter no global
filt_noglobal = 'roi_timeseries/_scan_rest_run-1/_selector_CSF-2mmE-M_aC-CSF+WM-2mm-DPC5_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz'
def get_roi(sub_folder):
    roi_folder = os.path.join(sub_folder,filt_noglobal)
    if os.path.exists(os.path.join(roi_folder, 'roi_stats.npz')):
        call(['unzip', os.path.join(roi_folder, 'roi_stats.npz'), '-d', roi_folder])
        # first row is nans and skip first 5
        roi = np.load(os.path.join(roi_folder, 'arr_0.npy'), allow_pickle=True)[5:] 
    else:
        print(f"Couldn't get rois for {sub_folder}")
        roi = None
    return roi

sub_51470_roi = get_roi(data_dir + '/cpac_output_single/output/pipeline_abide_cpac_run_freq-filter_nuisance/sub-0051470_ses-1')

In [218]:
# run in same pipeline twice
sub_51470_roi_2 = subjects_d[51470]._data_dict['cpac_filt_noglobal'] 
dist = np.linalg.norm(sub_51470_roi-sub_51470_roi_2)
dist

112.76785058445424

In [9]:
cpac_output_subs = list()
for sub, s in subjects_d.items():
    if 'cpac_filt_noglobal_t' in s._data_dict:
        cpac_output_subs.append(s)
print(len(cpac_output_subs))

10


In [19]:
for s in cpac_output_subs:
    cpac_roi = s._data_dict['cpac_filt_noglobal_t']
    # cpac_roi = s._data_dict['nofilt_global_roi_200_Cradd']
    preprocessed_roi = s._data_dict['filt_noglobal_roi_200_Cradd']
    dist = np.linalg.norm(cpac_roi-preprocessed_roi)
    print(dist)

2602.7420675701264
3000.534796103486
2311.5800366862463
3514.131042611196
1693.2367400830904
2029.1423662355703
2513.026675380683
2009.1285979719635
1931.0890611669913
2353.0316024266267


In [7]:
for i, s in enumerate(cpac_output_subs):
    print(s._sub_id)
    # roi_1 = s._data_dict['cpac_filt_noglobal']
    roi_1 = s._data_dict['filt_noglobal_roi_200_Cradd']
    s2 = cpac_output_subs[i + 1]
    # roi_2 = s2._data_dict['cpac_filt_noglobal']
    roi_2 = s2._data_dict['filt_noglobal_roi_200_Cradd']
    dist =  dist = np.linalg.norm(roi_2-roi_1)
    print(dist)

50653


ValueError: operands could not be broadcast together with shapes (176,200) (236,200) 